# Transfer Learning CIFAR10

In [4]:
%tensorflow_version 2.x

import tensorflow as tf
print(tf.__version__)

import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

2.1.0


* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [5]:
(X_train, y_train), (X_test, y_test) =  tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [6]:
print(f'There are {X_train.shape[0]} train images and {X_test.shape[0]} test images.')
print(f'There are {np.unique(y_train).shape[0]} unique classes to predict.')

There are 50000 train images and 10000 test images.
There are 10 unique classes to predict.


In [0]:
y_train_new = []
y_test_new = []

for i in y_train:
   y_train_new.append(i[0])

for i in y_test:
  y_test_new.append(i[0])

y_train = np.array(y_train_new)
y_test = np.array(y_test_new)

In [0]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_train2 = X_train[y_train >= 5]
y_train2 = y_train[y_train >= 5]


X_test1= X_test[y_test < 5]
y_test1= y_test[y_test < 5]
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5]

In [9]:
print("first train data set shape",X_train1.shape)
print("first test data set shape",X_test1.shape)

print("second train data set shape",X_train2.shape)
print("second test data set shape",X_test2.shape)

first train data set shape (25000, 32, 32, 3)
first test data set shape (5000, 32, 32, 3)
second train data set shape (25000, 32, 32, 3)
second test data set shape (5000, 32, 32, 3)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
num_classes = 5

y_train1 = tf.keras.utils.to_categorical(y_train1, num_classes)
y_test1 = tf.keras.utils.to_categorical(y_test1, num_classes)

In [0]:
X_train1 = X_train1/255
X_test1 = X_test1/255

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [12]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',name='conv1',input_shape=(32, 32, 3)))
model.add(tf.keras.layers.Conv2D(128, (3, 3),name= 'conv2', activation='relu'))


model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512,name='dense1', activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(100,name='dense2', activation='relu'))

model.add(tf.keras.layers.Dense(num_classes, activation='softmax',name='dense3'))
model.summary()


model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adamax')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 30, 30, 32)        896       
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 128)       36992     
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense1 (Dense)               (None, 512)               1

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [13]:
model.fit(X_train1, y_train1 ,validation_data=[X_test1,y_test1], epochs=50,batch_size=64)

Train on 25000 samples, validate on 5000 samples
Epoch 1/50
25000/25000 [==============================] - 14s 561us/sample - loss: 0.9934 - accuracy: 0.6155 - val_loss: 1.2705 - val_accuracy: 0.4640
Epoch 2/50
25000/25000 [==============================] - 6s 240us/sample - loss: 0.6593 - accuracy: 0.7480 - val_loss: 0.7723 - val_accuracy: 0.7076
Epoch 3/50
25000/25000 [==============================] - 6s 238us/sample - loss: 0.4621 - accuracy: 0.8280 - val_loss: 0.6316 - val_accuracy: 0.7688
Epoch 4/50
25000/25000 [==============================] - 6s 240us/sample - loss: 0.3089 - accuracy: 0.8885 - val_loss: 0.6343 - val_accuracy: 0.7754
Epoch 5/50
25000/25000 [==============================] - 6s 239us/sample - loss: 0.1878 - accuracy: 0.9376 - val_loss: 0.6544 - val_accuracy: 0.7808
Epoch 6/50
25000/25000 [==============================] - 6s 239us/sample - loss: 0.1146 - accuracy: 0.9652 - val_loss: 0.6910 - val_accuracy: 0.7890
Epoch 7/50
25000/25000 [==========================

In [14]:
loss, accuracy = model.evaluate(X_test1,y_test1)
print(f'Final Loss: {loss}, Final Accuracy: {accuracy}')

5000/5000 [==============================] - 1s 121us/sample - loss: 1.0976 - accuracy: 0.8066
Final Loss: 1.0975693259239196, Final Accuracy: 0.8065999746322632


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
X_train2 = X_train2/255
X_test2 = X_test2/255

y_train2 = y_train2-5
y_test2 = y_test2-5

y_train2 = tf.keras.utils.to_categorical(y_train2, num_classes)
y_test2 = tf.keras.utils.to_categorical(y_test2, num_classes)

In [16]:
model2 = tf.keras.Sequential()
trainableList = ['dense1','dense2','dense3']

for layer in model.layers:
    if(layer.name in trainableList):
       layer.trainable = True
    else:
       layer.trainable = False
    model2.add(layer)

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 30, 30, 32)        896       
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 128)       36992     
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense1 (Dense)               (None, 512)              

In [0]:
model2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adamax')

In [18]:
model2.fit(X_train2, y_train2 ,validation_data=[X_test2,y_test2], epochs=50,batch_size=64)

Train on 25000 samples, validate on 5000 samples
Epoch 1/50
25000/25000 [==============================] - 4s 166us/sample - loss: 0.8556 - accuracy: 0.7438 - val_loss: 0.4779 - val_accuracy: 0.8236
Epoch 2/50
25000/25000 [==============================] - 4s 141us/sample - loss: 0.3058 - accuracy: 0.8905 - val_loss: 0.4122 - val_accuracy: 0.8506
Epoch 3/50
25000/25000 [==============================] - 4s 142us/sample - loss: 0.1655 - accuracy: 0.9420 - val_loss: 0.4172 - val_accuracy: 0.8644
Epoch 4/50
25000/25000 [==============================] - 3s 138us/sample - loss: 0.0852 - accuracy: 0.9720 - val_loss: 0.4531 - val_accuracy: 0.8628
Epoch 5/50
25000/25000 [==============================] - 4s 141us/sample - loss: 0.0449 - accuracy: 0.9870 - val_loss: 0.4705 - val_accuracy: 0.8706
Epoch 6/50
25000/25000 [==============================] - 4s 141us/sample - loss: 0.0289 - accuracy: 0.9911 - val_loss: 0.5062 - val_accuracy: 0.8686
Epoch 7/50
25000/25000 [===========================

In [19]:
loss, accuracy = model2.evaluate(X_test2,y_test2)
print(f'Final Loss: {loss}, Final Accuracy: {accuracy}')

5000/5000 [==============================] - 1s 110us/sample - loss: 0.8180 - accuracy: 0.8796
Final Loss: 0.8180453688293695, Final Accuracy: 0.8795999884605408


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [22]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [24]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [25]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [26]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [0]:
y_train_tfidf = twenty_train.target
y_test_tfidf = twenty_test.target

X_train = twenty_train.data
X_test = twenty_test.data

In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),stop_words='english',min_df = 2)

In [30]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
print(X_train_tfidf.shape)

X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(X_test_tfidf.shape)

(2257, 65243)
(1502, 65243)


In [38]:
print("feature",tfidf_vectorizer.get_feature_names())

feature ['00', '00 00', '00 08', '00 09', '00 10', '00 34', '00 35', '00 57', '00 58', '00 59', '00 did', '00 gmt', '00 p003228', '00 pm', '00 thank', '000', '000 000', '000 bytes', '000 cash', '000 civil', '000 colors', '000 inquistions', '000 iu', '000 massacares', '000 people', '000 pounds', '000 question', '000 riots', '000 times', '000 year', '000 years', '0000001200', '0000001200 355o33', '0001', '0001 south', '0010580b', '0010580b vmcbrt', '001642', '001642 9186', '002', '003258u19250', '003258u19250 uicvm', '00pm', '01', '01 02', '01 08', '01 09', '01 10', '01 11', '01 19', '01 24', '01 53', '01 55', '01 56', '01 differences', '01 doesn', '01 lines', '01 v2', '0100', '0100 mathew', '011720', '011720 28958', '0138', '0138 lehigh', '01580', '01580 change', '0184', '02', '02 04', '02 1993', '02 22', '02 31', '02 321006', '02 34', '02 39', '02 42', '02 50', '02 51', '02 52', '02 53', '02 54', '02 55', '02 56', '02 edt', '02 reputable', '0200', '0200 australia', '020504', '020504 19

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [39]:
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train_tfidf)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
acc_score = lr_model.score(X_train_tfidf,y_train_tfidf) 
print("Train Accuracy Score: ", acc_score)

Train Accuracy Score:  0.9964554718653079


In [42]:
acc_score = lr_model.score(X_test_tfidf,y_test_tfidf)
print("test Accuracy Score: ", acc_score)

test Accuracy Score:  0.9087882822902796
